# MBTI - 16 Personalities classifier

# Data acquisition

## MBTI Dataset
The **MBTI** (Myers-Briggs Personality Type Indicator) divides everyone into 16 distinct personality types across 4 axis:
1. Introversion (I) – Extroversion (E)
2. Intuition (N) – Sensing (S)
3. Thinking (T) – Feeling (F)
4. Judging (J) – Perceiving (P)

This system is used in _businesses, online,_ for _fun,_ for _research_ and lots more.

This **dataset** contains over *8600 rows of data*, on each row is a person’s:
* Type (4 letter MBTI code/type)
* A section of each of the last 50 things they have posted (Each entry separated by "|||" (3 pipe characters))

## Acknowledgements
This data was collected through the PersonalityCafe forum, as it provides a large selection of people and their MBTI personality type, as well as what they have written.

## Downloading the dataset onto the cluster

I hosted the dataset on my [github](https://github.com/edu-rinaldi/MBTI-Predictor/blob/main/dataset/mbti_1.csv.zip), so that I and whoever want to train a different model can download it in an easier way.

In [0]:
%sh wget -P /tmp https://github.com/edu-rinaldi/MBTI-Predictor/raw/main/dataset/mbti_1.csv

--2021-05-10 13:00:38-- https://github.com/edu-rinaldi/MBTI-Predictor/raw/main/dataset/mbti_1.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/edu-rinaldi/MBTI-Predictor/main/dataset/mbti_1.csv [following]
--2021-05-10 13:00:38-- https://raw.githubusercontent.com/edu-rinaldi/MBTI-Predictor/main/dataset/mbti_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62856486 (60M) [text/plain]
Saving to: ‘/tmp/mbti_1.csv’

 0K .......... .......... .......... .......... .......... 0% 7.40M 8s
 50K .......... .......... .......... .......... .......... 0% 7.48M 8s
 100K .......... .......... .......... .......... .......... 0% 8.10M 8s
 150K .......... .......... .......... .......... .......... 0% 15.4M 7s
 200K .......... .......... .......... .......... .......... 0% 10.6M 7s
 250K .......... .......... .......... .......... .......... 0% 12.6M 6s
 300K .......... .......... .......... .......... .......... 0% 16.6M 6s
 350K .......... .......... .......... .......... .......... 0% 12.6M 6s
 400K .......... .......... .......... .......... .......... 0% 11.7M 6s
 450K .......... .......... .......... .......... .......... 0% 24.4M 5s
 500K .......... .......... .......... .......... .......... 0% 11.7M 5s
 550K .......... .......... .......... .......... .......... 0% 17.0M 5s
 600K .......... .......... .......... .......... .......... 1% 12.8M 5s
 650K .......... .......... .......... .......... .......... 1% 17.9M 5s
 700K .......... .......... .......... .......... .......... 1% 20.3M 5s
 750K .......... .......... .......... .......... .......... 1% 17.1M 5s
 800K .......... .......... .......... .......... .......... 1% 16.9M 5s
 850K .......... .......... .......... .......... .......... 1% 17.7M 5s
 900K .......... .......... .......... .......... .......... 1% 19.2M 5s
 950K .......... .......... .......... .......... .......... 1% 15.5M 4s
 1000K .......... .......... .......... .......... .......... 1% 16.8M 4s
 1050K .......... .......... .......... .......... .......... 1% 17.9M 4s
 1100K .......... .......... .......... .......... .......... 1% 37.3M 4s
 1150K .......... .......... .......... .......... .......... 1% 26.5M 4s
 1200K .......... .......... .......... .......... .......... 2% 14.6M 4s
 1250K .......... .......... .......... .......... .......... 2% 54.0M 4s
 1300K .......... .......... .......... .......... .......... 2% 17.9M 4s
 1350K .......... .......... .......... .......... .......... 2% 21.5M 4s
 1400K .......... .......... .......... .......... .......... 2% 19.8M 4s
 1450K .......... .......... .......... .......... .......... 2% 16.3M 4s
 1500K .......... .......... .......... .......... .......... 2% 44.3M 4s
 1550K .......... .......... .......... .......... .......... 2% 18.2M 4s
 1600K .......... .......... .......... .......... .......... 2% 22.0M 4s
 1650K .......... .......... .......... .......... .......... 2% 21.4M 4s
 1700K .......... .......... .......... .......... .......... 2% 30.9M 4s
 1750K .......... .......... .......... .......... .......... 2% 27.6M 4s
 1800K .......... .......... .......... .......... .......... 3% 18.4M 4s
 1850K .......... .......... .......... .......... .......... 3% 36.7M 4s
 1900K .......... .......... .......... .......... .......... 3% 29.8M 3s
 1950K .......... .......... .......... .......... .......... 3% 24.4M 3s
 2000K .......... .......... .......... .......... .......... 3% 23.0M 3s
 2050K .......... .......... .......... .......... .......... 3% 62.2M 3s
 2100K .......... .......... .......... .......... .......... 3% 18.9M 3s
 2150K ........

In [0]:
dbutils.fs.mv("file:/tmp/mbti_1.csv", "/bdc-2020-21/datasets/")

Out[19]: True

In [0]:
%fs ls /bdc-2020-21/datasets/

path,name,size
dbfs:/bdc-2020-21/datasets/AllProductReviews.csv,AllProductReviews.csv,2496851
dbfs:/bdc-2020-21/datasets/mbti_1.csv,mbti_1.csv,62856486
dbfs:/bdc-2020-21/datasets/mnist-train.csv.bz2,mnist-train.csv.bz2,6732270


# Importing Spark (and some other cool things)
Python's imports

In [0]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Now let's check if everything is ok

In [0]:
spark

Out[2]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

# Data analysis

Now that we have downloaded the dataset and we have imported Spark framework, we can start analyzing the dataset... but first let's load the dataset into a spark dataframe:

In [0]:
mbti_df = spark.read.load("dbfs:/bdc-2020-21/datasets/mbti_1.csv", 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                         )

In [0]:
mbti_df.show(5)

+----+--------------------+
type| posts|
+----+--------------------+
INFJ|'http://www.youtu...|
ENTP|'I'm finding the ...|
INTP|'Good one _____ ...|
INTJ|'Dear INTP, I e...|
ENTJ|'You're fired.|||...|
+----+--------------------+
only showing top 5 rows

...TBC...